In [2]:
from pathlib import Path
from tqdm import tqdm

import pandas as pd
import geopandas as gpd
import xarray as xr
import torch

from src.cat_interp import CatchmentInterpolator
from src.schedule import define_schedule
from src.models import MultiStageLearnedMLP
from src.utils import nse_loss_fn

In [3]:
from src.config import DATA_ROOT as root

### Helpers

In [4]:
from src.geoglow_io import *
from src.schedule import define_schedule
from src.cat_interp import CatchmentInterpolator
from diffroute import get_node_idxs

def load_vpu(vpu, device):
    all_discharges = pd.read_feather(root / "retro_feather" / f"{vpu}.feather")
    gdf = pd.read_feather(root / "tdxhydro_feather" / f"streams_{vpu}.feather")

    g, runoff, interp_df = load_geoflow(vpu_numbers=[vpu])
    annotate_graph_with_physical_prop(g, gdf)
    nodes_idx = get_node_idxs(g)
    runoff = runoff.loc[all_discharges.index]
    discharges = all_discharges[nodes_idx.index]

    clusters_g, node_transfer = define_schedule(g, plength_thr=10**4, node_thr=20000)
    cat = CatchmentInterpolator(clusters_g, runoff, interp_df, device=device)
    return g, clusters_g, node_transfer, cat, discharges

def test(ms, discharges, N):
    df_out = []
    df_y = []
    nses = []
    
    with torch.no_grad():
        outputs = ms.forward(N)
        for i,output in enumerate(tqdm(outputs)):
            out = pd.DataFrame(output.cpu().squeeze(), index=ms.node_idxs[i].index).T
            y = discharges[out.columns].reset_index(drop=True)
            nse = 1-(((out-y)**2).mean() / y.var())
    
            nses+=[nse]
            df_out += [out]
            df_y += [y]
    return nses, df_out, df_y

#### params

In [5]:
model_name="muskingum"
max_delay = 50
dt = 1/24
epochs = 1000
device = "cuda:3"
test_device = "cuda:4" # GPU memory gets full so test on another GPU
n_iter = 100

#### Init

In [ ]:
g, clusters_g, node_transfer, cat, discharges = load_vpu("603", device)
g2, clusters_g2, node_transfer2, cat2, discharges2 = load_vpu("602", test_device)

In [7]:
stds = read_physical_prop(g)[-1] # Important: Need to normalize physical properties using the same std.
ms = MultiStageLearnedMLP(clusters_g, node_transfer, cat, param_stds=stds).to(device)
ms.init_layers_buffers(device)
ms2 = MultiStageLearnedMLP(clusters_g2, node_transfer2, cat2, param_stds=stds).to(test_device)
ms2.init_layers_buffers(test_device)

opt = torch.optim.Adam(ms.parameters(), lr=.0001)
ys = [torch.from_numpy(discharges[nodes_idx.index].values.T)[None].to(device) 
     for nodes_idx in tqdm(ms.node_idxs)]

  0%|          | 0/52 [00:00<?, ?it/s]
 [00:00, ?it/s]
100%|██████████| 52/52 [00:00<00:00, 150.30it/s]
52it [00:00, 150.55it/s]
  0%|          | 0/17 [00:00<?, ?it/s]
 [00:00, ?it/s]
100%|██████████| 17/17 [00:00<00:00, 122.08it/s]
17it [00:00, 123.67it/s]


  0%|          | 0/52 [00:00<?, ?it/s]

#### Training

In [ ]:
all_losses = []
res = []

for N in tqdm(range(1,18)):
    ms.mlp.to(device)
    for i in tqdm(range(n_iter)):
        outputs = ms.forward(N)
        losses = [nse_loss_fn(out.squeeze(), y.squeeze()) \
                  for out, y in zip(outputs[:N], ys[:N])]
        loss = sum(losses) / len(losses)
        
        
        loss.backward()
        opt.step()
        opt.zero_grad()
        
        all_losses.append(loss.item())
        print((loss.item()))#, [x.item() for x in losses]))
        
    nses, df_out, df_y = test(ms, discharges, N=1)
    print([x.mean() for x in nses])
    print([x.median() for x in nses])

    for model in ms2.routers: model.mlp =  ms.mlp.to(test_device)
    tnses, df_out, df_y = test(ms2, discharges2, N=1)
    print([nse.median() for nse in tnses])
    print([nse.mean() for nse in tnses])
    res.append((nses, tnses))

  0%|          | 0/100 [00:00<?, ?it/s]

2.242368221282959
2.1881749629974365
2.133002519607544
2.0766680240631104
2.019174814224243
1.9604743719100952
1.9006462097167969
1.8397632837295532
1.7778904438018799
1.7151025533676147
1.6514334678649902
1.5869015455245972
1.5216141939163208
1.4556466341018677
1.3890966176986694
1.322152853012085
1.254896879196167
1.1875550746917725
1.1204158067703247
1.053498387336731
0.9869477152824402
0.920948326587677
0.8557178378105164
0.7914872765541077
0.7285439968109131
0.667191207408905
0.6077373623847961
0.5504348278045654
0.49557480216026306
0.4433974325656891
0.3942476511001587
0.348408967256546
0.3060794472694397
0.26745766401290894
0.23268622159957886
0.20182795822620392
0.17487025260925293
0.15172916650772095
0.13227182626724243
0.1163206398487091
0.10362468659877777
0.0939001590013504
0.08682490885257721
0.0820600837469101
0.07924021035432816
0.07799863070249557
0.07798223942518234
0.0788537785410881
0.08031033724546432
0.08208249509334564
0.08394575119018555
0.08571714162826538
0.087

  0%|          | 0/1 [00:00<?, ?it/s]

[0.9944226]
[0.9974633]


  0%|          | 0/1 [00:00<?, ?it/s]

...

[0.9980698]
[0.9992689]


  0%|          | 0/1 [00:00<?, ?it/s]

[0.99932474]
[0.9982877]


  0%|          | 0/100 [00:00<?, ?it/s]

0.051817383617162704


### Eval

In [9]:
for model in ms2.routers: model.mlp =  ms.mlp.to(test_device)
tnses, df_out, df_y = test(ms2, discharges2, N=N)
# Check if any NSE disparity between cluster: minimal
for nse, tnse in res: print([x.median() for x in tnse])

  0%|          | 0/12 [00:00<?, ?it/s]

[0.99803555]
[0.9989303]
[0.99920183]
[0.99932474]
[0.9995174]
[0.9996587]
[0.99973077]
[0.9998072]
[0.99982846]
[0.9998453]
[0.9998565]


In [15]:
pd.concat(tnses).median()

0.9997825